In [1]:
!pip install groq langchain chromadb faiss-cpu flask fastapi uvicorn pypdf



     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 8.2 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 131.4/131.4 kB 11.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.8/19.8 MB 89.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.4/31.4 MB 23.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.5/310.5 kB 34.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 284.2/284.2 kB 29.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 62.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.3/103.3 kB 12.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.5/16.5 MB 98.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.6/65.6 kB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.5/72.5 kB 8.3 MB/s eta

In [12]:
import os
os.environ["GROQ_API_KEY"] = "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"


In [2]:
!pip install PyPDF2


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 16.2 MB/s eta 0:00:00


In [3]:
import os
from PyPDF2 import PdfReader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from google.colab import files


uploaded = files.upload()
pdf_files = list(uploaded.keys())

all_text = ""
for pdf in pdf_files:
    reader = PdfReader(pdf)
    for page in reader.pages:
        all_text += page.extract_text() + "\n"

print("Extracted text length:", len(all_text))


text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=200,
    length_function=len
)

chunks = text_splitter.split_text(all_text)

print(f" Total Chunks Created: {len(chunks)}")
print("Sample chunk:\n", chunks[0][:500])


Saving Hands-On_Machine_Learning_with_Scikit-Learn-Keras-and-TensorFlow-2nd-Edition-Aurelien-Geron.pdf to Hands-On_Machine_Learning_with_Scikit-Learn-Keras-and-TensorFlow-2nd-Edition-Aurelien-Geron.pdf
Extracted text length: 1710819
 Total Chunks Created: 2156
Sample chunk:
 Aurélien GéronHands-on  
Machine Learning   with Scikit-Learn,  
Keras & TensorFlow
Concepts, Tools, and Techniques  
to Build Intelligent Systems
TM
2nd Edition
Updated for  TensorFlow 2


In [4]:
!pip install -U langchain-community sentence-transformers


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 55.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.2/45.2 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 6.0 MB/s eta 0:00:00


In [13]:
import re


clean_chunks = []
for c in chunks:
    if isinstance(c, str):
        text = re.sub(r'[^a-zA-Z0-9\s.,!?;:()\-\']+', ' ', c)
        clean_chunks.append(text.strip())

print("Cleaned Chunks:", len(clean_chunks))


Cleaned Chunks: 2156


In [6]:
from langchain_community.vectorstores import Chroma
from langchain_community.embeddings import HuggingFaceEmbeddings


embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/paraphrase-MiniLM-L3-v2")


persist_directory = "chroma_db"

vectordb = Chroma.from_texts(
    texts=clean_chunks,
    embedding=embedding_model,
    persist_directory=persist_directory
)

vectordb.persist()
print("ChromaDB created and embeddings stored with", len(clean_chunks), "chunks")


/tmp/ipython-input-1317827797.py:5: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/paraphrase-MiniLM-L3-v2")
/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access publi

modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

ChromaDB created and embeddings stored with 2156 chunks


/tmp/ipython-input-1317827797.py:16: LangChainDeprecationWarning: Since Chroma 0.4.x the manual persistence method is no longer supported as docs are automatically persisted.
  vectordb.persist()


In [14]:
from groq import Groq

client = Groq(api_key=os.environ["GROQ_API_KEY"])

def ask_question(query, k=3):
    """
    query: user question
    k: number of top chunks to retrieve
    """

    docs = vectordb.similarity_search(query, k=k)
    context = "\n\n".join([d.page_content for d in docs])


    prompt = f"""
    You are an AI assistant. Use the following context from documents to answer the question.

    Context:
    {context}

    Question: {query}

    Answer:
    """

    chat_completion = client.chat.completions.create(
        messages=[{"role": "user", "content": prompt}],
        model="llama-3.1-8b-instant",
    )

    return chat_completion.choices[0].message.content



question = "What is ?"
answer = ask_question(question)
print("Q:", question)
print("A:", answer)


Q: What is ?
A: Based on the provided context, I will try to reasonably guess the meaning of the term that is missing.

From the text, several terms and concepts are mentioned, such as:

1. Scaled Dot-Product Attention layer
2. Encoder
3. Decoder
4. Vectorized representations
5. Dense layer
6. Positional encodings
7. Multi-Head Attention layer
8. Subword tokenizers
9. SentencePiece project
10. Neural Machine Translation

However, I notice that the text refers to a specific architecture but does not mention which particular architecture it is discussing. The text talks about the encoder, decoder, and attention layers, which are components of a sequence-to-sequence model, particularly a Transformer.

Given that you've provided the context of papers and projects related to neural text processing, including the Google's SentencePiece project, Taku Kudo and John Richardson's paper on SentencePiece, and Rico Sennrich et al.'s paper on Neural Machine Translation of Rare Words with Subword Uni

In [15]:
!pip install fastapi uvicorn pyngrok


In [16]:
code = """
import os
import re
from fastapi import FastAPI, UploadFile, Form
from fastapi.responses import JSONResponse
from PyPDF2 import PdfReader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import Chroma
from langchain_community.embeddings import HuggingFaceEmbeddings
from groq import Groq
import shutil

app = FastAPI()
persist_directory = "chroma_db"

# Embeddings
embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/paraphrase-MiniLM-L3-v2")

# Vector DB (reload if exists)
vectordb = Chroma(persist_directory=persist_directory, embedding_function=embedding_model)

# Groq client
client = Groq(api_key=os.environ["GROQ_API_KEY"])

def process_pdf(pdf_path):
    reader = PdfReader(pdf_path)
    text = ""
    for page in reader.pages:
        text += page.extract_text() + "\\n"

    text = re.sub(r'[^a-zA-Z0-9\\s.,!?;:()\\-\\']+', ' ', text)
    splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
    return splitter.split_text(text)

@app.post("/upload")
async def upload(file: UploadFile):
    path = f"temp_{file.filename}"
    with open(path, "wb") as buffer:
        shutil.copyfileobj(file.file, buffer)

    chunks = process_pdf(path)
    vectordb.add_texts(chunks)
    vectordb.persist()
    return {"status": "success", "chunks_added": len(chunks)}

@app.post("/query")
async def query(question: str = Form(...)):
    docs = vectordb.similarity_search(question, k=3)
    context = "\\n\\n".join([d.page_content for d in docs])

    prompt = f\"\"\"Use the following context from documents to answer:

    Context:
    {context}

    Question: {question}

    Answer:
    \"\"\"

    chat_completion = client.chat.completions.create(
        messages=[{"role": "user", "content": prompt}],
        model="llama-3.1-8b-instant",
    )
    return {"question": question, "answer": chat_completion.choices[0].message.content}

@app.get("/metadata")
async def metadata():
    return {"documents": vectordb._collection.count()}
"""

with open("main.py", "w") as f:
    f.write(code)

print("main.py created successfully")


main.py created successfully


In [17]:
!pip install gradio


In [18]:
import gradio as gr

def rag_query(question):
    docs = vectordb.similarity_search(question, k=3)
    context = "\n\n".join([d.page_content for d in docs])
    prompt = f"""
    Use the following context from documents to answer:

    Context:
    {context}

    Question: {question}

    Answer:
    """
    chat_completion = client.chat.completions.create(
        messages=[{"role": "user", "content": prompt}],
        model="llama-3.1-8b-instant",
    )
    return chat_completion.choices[0].message.content


demo = gr.Interface(
    fn=rag_query,
    inputs=gr.Textbox(label="Ask a question about your PDFs"),
    outputs="text",
    title=" RAG with Groq API",
    description="Upload your PDFs first, then ask questions!"
)

demo.launch(share=True)


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://f9f593ca46ff84a9dd.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
